In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from multiprocessing import Pool, cpu_count
import concurrent.futures
from tqdm import tqdm  # 导入 tqdm 库
import json
import os
from sentence_transformers import SentenceTransformer, SimilarityFunction



/scratch/Wu/miniconda3/envs/openai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

/scratch/Wu/miniconda3/envs/openai/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/scratch/Wu/miniconda3/envs/openai/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:


stride = 512
max_length = 512
prev_end_loc = 0

def getperplexity(sentence):
    nlls = []
    prev_end_loc = 0
    encodings = tokenizer(sentence, return_tensors='pt')
    seq_len = encodings.input_ids.size(1)

    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    ppl = torch.exp(torch.stack(nlls).mean())
sentence = 'sss buka first'
print(getperplexity(sentence))

In [ ]:
sentence = 'demo here,我吉三不臣杨点拉录perplexity！'
encodings = tokenizer(sentence, return_tensors='pt')
stride = 512
seq_len = encodings.input_ids.size(1)
seq_len
max_length = 512
prev_end_loc = 0

In [13]:
def getp (sentence):
    encodings = tokenizer(sentence, return_tensors='pt')
    stride = 512
    seq_len = encodings.input_ids.size(1)
    max_length = 512
    prev_end_loc = 0
    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    ppl = float(torch.exp(torch.stack(nlls).mean()))
    print(ppl)
    return ppl

In [14]:
sentence = 'test here !!'
getp(sentence)

70419.515625


70419.515625

In [15]:
with open('result_1/one_1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
def process_item(item):
    item['sentence_length'] = len(item['evol_prompt'])
    item['perplexity'] = getp(item['evol_prompt'])
    return item
# for item in data:
#     item['sentence_length'] = len(item['evol_prompt'])
#     item['perplexity'] = getp(item['evol_prompt'])

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 使用 tqdm 包装 data 列表以显示进度条
    results = list(tqdm(executor.map(process_item, data), total=len(data)))

In [ ]:
results

In [17]:
with open('result_1.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)


In [20]:
# 存储 JSON 文件名的列表
folder_path = 'result_1'
json_filenames = []

# 遍历文件夹中的文件
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):  # 检查文件扩展名是否为 .json
        json_filenames.append(filename)

json_filenames

['one_1.json',
 'four_2_2.json',
 'four_1_2.json',
 'four_2_1.json',
 'seven_7_4.json',
 'four_1_1.json',
 'four_3_2.json',
 'four_3_1.json',
 'four_4_1.json',
 'six_1.json',
 'four_2_3.json']

In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")
model.similarity_fn_name = SimilarityFunction.COSINE

real_sentence = '优点：爱学习，看书，1000~2000本书，90%都看过。市场，销售财务。看书比较杂，人文。国家形式，全球的形式\
1. 对商业的趋势判断，很感兴趣。\
2. 安静的时候，能量更大。\
----中----\
多把学到应用出来，不能再看书了。要开始读无字书（人）\
卢总前两-三年都是在反腐，我管辖范围内也有出问题的。\
---待提升---\
心理素质：有时候沉不住气\
更喜欢一个人待着\
'
real_embedding = model.encode(real_sentence)

/scratch/Wu/miniconda3/envs/openai/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def getsimilarity(sentence):
    embedding = model.encode(sentence)
    similarities = model.similarity(real_embedding, embedding)
    return similarities
